In [1]:
import pandas as pd
import re
import numpy as np
from src import functions
import datetime

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x000001F3CB07F8C8; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


In [3]:
df = functions.base(cur)

In [4]:
df_altas = functions.base_altas(df)

## Ventas

In [4]:
# functions.ventas(df)

In [5]:
# functions.altas(df,df_altas )

In [7]:
# functions.bajas(df,df_altas)

In [8]:
mig_df=functions.migBase(cur)

In [23]:
migin, migout = functions.mig_producto_mes(mig_df)

In [25]:
migin.reset_index(inplace=True)
migin['index'] = migin['canal_venta']+'_'+migin['product_name']
migin.drop(columns=['canal_venta','Family','product_name'],inplace=True)
migin.set_index(['index'],inplace=True)

In [26]:
migin

,level_0,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
index,,,,,,,,,,,,,,,,,
DHO_X Fiber 100 - neba,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DHO_Teamwork+PBX,1,0,0,0,0,0,0,0,11,0,2,1,1,3,1,0,1
IT integrator_Teamwork+PBX,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [33]:
getVelocityFrom('20')

'Portable 20'